What topics are related to Finland in the New York Times in the past 10 years.

In [10]:
import json

data = json.load( open('./examples/ny_times.json') )
data = map( lambda x: x['abstract'], data ) ## only select the abstracts

data = filter( lambda x: x != None, data )

print len( data ), 'abstracts'

531 abstracts


In [16]:
## stem content

print data[0]

import nltk
import nltk.data

lemma = nltk.stem.wordnet.WordNetLemmatizer()

from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    
    return wordnet.NOUN ## default to

data2 = []

for d in data:
    
    words = nltk.word_tokenize( d )
    
    dd = ''

    for tagged_word in nltk.pos_tag( words ):
        word = tagged_word[0]
        tag = tagged_word[1]
        dd += lemma.lemmatize( word , get_wordnet_pos( tag ) ) + ' '
        
    data2.append( dd )
    
print data2[0]
    
json.dump( data2 , open('nyt_times_stemmed.json', 'w') )

print "Done"

Microsoft’s purchase of some of Nokia was marked by a series of high-level negotiations that included Steven A. Ballmer that went nowhere.
Microsoft’s purchase of some of Nokia be mark by a series of high-level negotiation that include Steven A. Ballmer that go nowhere . 
Done


In [34]:
texts = json.load( open('nyt_times_stemmed.json') )

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

## create document term matrix
counter = CountVectorizer( max_df=.20, min_df=5 )
dtm = counter.fit_transform( texts ).toarray()
labels = np.array( counter.get_feature_names() )

logll = {}
models = {}

## fit different numbers of topics
for k in range(2, 26):
    
    model = LatentDirichletAllocation( n_topics= k , max_iter=5, learning_method='batch', learning_offset=50., random_state=0 )
    
    model.fit( dtm )
    ll = model.score( dtm ) ## TODO: see if there's better measurement
    
    logll[k] = ll
    models[k] = model

## find the best fit
m = logll[2]
m_ind = 2

for i, value in logll.items():
    
    if value > m:
        m = value
        m_ind = i

print "Let's go with",m_ind,"topics"
 
model = models[ m_ind ]

## terms per topic
for topic_idx, topic in enumerate(model.components_):
    
    print "Topic #%d:" % topic_idx
    print " ".join( [labels[i].encode('utf8') for i in topic.argsort()[:-5 - 1:-1]])
    print
        
## topics per files

for i, topic in enumerate( model.transform( dtm ) ):
    
    topic = np.argmax( topic )    
    print topic, texts[i]

Let's go with 3 topics
Topic #0:
world win hockey defeat at

Topic #1:
that it have say from

Topic #2:
million percent have euro with

1 Microsoft’s purchase of some of Nokia be mark by a series of high-level negotiation that include Steven A. Ballmer that go nowhere . 
2 Rajeev Suri , Nokia’s 48-year-old chief executive , say his company’s $ 16.6 billion takeover of Alcatel-Lucent be exactly what Nokia need to execute it yearslong makeover , which have include layoff and the sale of unwanted asset . 
1 In some way , Jussi Leinonen’s prize-winning photograph come about by luck . But it teach him something important about photography : keep try . 
2 The Finnish gaming company behind the Clash of Clans say revenue in 2013 rise nearly ninefold compare with 2012 , and that pre-tax profit rise to $ 464 million from $ 51 million . 
0 Finland rout United States , 5-0 , to win the bronze medal in men’s hockey at the Sochi Games ; Finland have win four medal in the last five Olympics , more th